### 数据解析
- 正则
- bs4
- xpath
- pyquery（自学）

### 正则解析
    单字符：
        . : 除换行以外所有字符
        [] ：[aoe] [a-w] 匹配集合中任意一个字符
        \d ：数字  [0-9]
        \D : 非数字
        \w ：数字、字母、下划线、中文
        \W : 非\w
        \s ：所有的空白字符包,括空格、制表符、换页符等等。等价于 [ \f\n\r\t\v]。
        \S : 非空白
    数量修饰：
        * : 任意多次  >=0
        + : 至少1次   >=1
        ? : 可有可无  0次或者1次
        {m} ：固定m次 hello{3,}
        {m,} ：至少m次
        {m,n} ：m-n次
    边界：
        $ : 以某某结尾 
        ^ : 以某某开头
    分组：
        (ab)  
    贪婪模式： .*
    非贪婪（惰性）模式： .*?

- 使用正则进行图片数据的批量解析爬取

- 如何爬取图片数据？
    - 方式1：基于requests
    - 方式2：基于urllib
        - urllib模块作用和requests模块一样，都是基于网络请求的模块。
            - 当requests问世后就迅速的替代了urllib模块

In [2]:
import requests
import urllib
headers = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'
}

In [3]:
#方式1：
img_url = 'https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=1312059974,1893880587&fm=11&gp=0.jpg'
response = requests.get(url=img_url,headers=headers)
img_data = response.content #content返回的是二进制形式的响应数据
with open('1.jpg','wb') as fp:
    fp.write(img_data)

In [4]:
#方式2：
img_url = 'https://ss1.bdstatic.com/70cFuXSh_Q1YnxGkpoWK1HF6hhy/it/u=1312059974,1893880587&fm=11&gp=0.jpg'
#可以直接对url发起请求且进行持久化存储
urllib.request.urlretrieve(img_url,'./2.jpg')

('./2.jpg', <http.client.HTTPMessage at 0x10486a5c0>)

- 上述两种爬取图片的操作不同之处是什么？
    - 使用urllib的方式爬取图片无法进行UA伪装，而requests的方式可以。

- 需求：爬取校花网中的图片数据
    - www.521609.com
    - 操作：需要将每一张图片的地址解析出来，然后对图片地址发起请求即可。

- 分析浏览器开发者工具中Elements和network这两个选项卡对应的页面源码数据有何不同之处？
    - Elements中包含的显示的页面源码数据为当前页面所有的数据加载完毕后对应的完整的页面源码数据（包含了动态加载数据）
    - network中显示的页面源码数据仅仅为某一个单独的请求对应的响应数据（不包含动态加载数据）
    - 结论：如果在进行数据解析的时候，一定是需要对页面布局进行分析，如果当前网站没有动态加载的数据就可以直接使用Elements对页面布局进行分析，否则只可以使用network对页面数据进行分析。

In [8]:
import re
import os

In [9]:
dirName = 'ImgLibs'
if not os.path.exists(dirName):
    os.mkdir(dirName)
    
#1.捕获到当前首页的页面源码数据
url = 'http://www.521609.com/qingchunmeinv/'
page_text = requests.get(url=url,headers=headers).text

#2.从当前获取的页面源码数据中解析出图片地址
ex = '<li>.*?<img src="(.*?)" width=.*?</li>'
img_src_list = re.findall(ex,page_text,re.S)
for src in img_src_list:
    src = 'http://www.521609.com'+src
    imgPath = dirName+'/'+src.split('/')[-1]
    urllib.request.urlretrieve(src,imgPath)
    print(imgPath,'下载成功！！！')

ImgLibs/11046303404-1-lp.jpg 下载成功！！！
ImgLibs/1152K12363-1-lp.jpg 下载成功！！！
ImgLibs/10I4561000-1-lp.jpg 下载成功！！！
ImgLibs/116234L914-1-lp.jpg 下载成功！！！
ImgLibs/12236009605-1-lp.jpg 下载成功！！！
ImgLibs/11I24425C-1-lp.jpg 下载成功！！！
ImgLibs/1093002H94-1-lp.jpg 下载成功！！！
ImgLibs/10910261217-1-lp.jpg 下载成功！！！
ImgLibs/22111113364.jpg 下载成功！！！
ImgLibs/13554311594.jpg 下载成功！！！
ImgLibs/20052013192.jpg 下载成功！！！
ImgLibs/1_194614R3.jpg 下载成功！！！
ImgLibs/13545GI62.jpg 下载成功！！！
ImgLibs/000F41S33.jpg 下载成功！！！
ImgLibs/09355G45c.jpg 下载成功！！！
ImgLibs/00343314205.jpg 下载成功！！！


- 数据解析的作用？
    - 用来实现聚焦爬虫。
- 网页中显示的数据都是存储在哪里？
    - 都是存储在html的标签中或者是标签的属性中
- 数据解析的通用原理是什么？
    - 指定标签的定位
    - 取出标签中存储的数据或者标签属性中的数据

### bs4
- bs4解析原理
    - 实例化一个BeautifulSoup的对象，且将待解析的页面源码数据加载到该对象中
    - 调用BeautifulSoup对象中相关方法或者属性进行标签定位和文本数据的提取
- 环境安装：
    - pip install lxml #解析器
    - pip install bs4
- BeautifulSoup对象的实例化：
    - BeautifulSoup(fp,'lxml')：用来将本地存储的html文档中的数据进行解析
    - BeautifulSoup(page_text，’lxml‘)：用来将互联网上请求到的页面源码数据进行解析
- 标签定位：
    - soup.tagName：只可以定位到第一次出现的tagName标签
    - soup.find('tagName',attrName='value'):属性定位
    - soup.findAll:跟find一样用作属性定位，只不过findAll返回的是列表
    - soup.select('选择器'):选择器定位
        - 类选择器
        - id选择器
        - 层级选择
            - 大于号:表示一个层级
            - 空格：表示多个层级
- 取数据
    - .text：返回的是该标签下所有的文本内容
    - .string:返回的是该标签直系的文本内容
- 取属性：
    - tag['attrName']

In [27]:
from bs4 import BeautifulSoup
fp = open('./test.html','r')
soup = BeautifulSoup(fp,'lxml')
soup.p
soup.find('div',class_='song')
soup.find('a',id='feng')
soup.findAll('a',id='feng')
soup.select('.tang')
soup.select('.tang li')
a_tag = soup.find('a',id='feng')
a_tag['href']

'http://www.haha.com'

In [25]:
div_tag = soup.find('div',class_='song')
div_tag.text

'\n李清照\n王安石\n苏轼\n柳宗元\n\nthis is span\n\t\t宋朝是最强大的王朝，不是军队的强大，而是经济很强大，国民都很有钱\n总为浮云能蔽日,长安不见使人愁\n\n'

- 爬取三国全篇内容：http://www.shicimingju.com/book/sanguoyanyi.html

In [28]:
main_url = 'http://www.shicimingju.com/book/sanguoyanyi.html'
page_text = requests.get(url=main_url,headers=headers).text
fp = open('./sanguo.txt','w',encoding='utf-8')
#数据解析：章节标题，详情页url，章节内容
soup = BeautifulSoup(page_text,'lxml')
#定位到的所有的符合要求的a标签
a_list = soup.select('.book-mulu > ul > li > a')
for a in a_list:
    title = a.string
    detail_url = 'http://www.shicimingju.com'+a['href']
    
    #对详情页发起请求解析出章节内容
    page_text_detail = requests.get(url=detail_url,headers=headers).text
    soup = BeautifulSoup(page_text_detail,'lxml')
    div_tag = soup.find('div',class_="chapter_content")
    content = div_tag.text
    fp.write(title+':'+content+'\n')
    print(title,'保存成功！！！')
fp.close()
    

第一回·宴桃园豪杰三结义  斩黄巾英雄首立功 保存成功！！！
第二回·张翼德怒鞭督邮    何国舅谋诛宦竖 保存成功！！！
第三回·议温明董卓叱丁原  馈金珠李肃说吕布 保存成功！！！
第四回·废汉帝陈留践位    谋董贼孟德献刀 保存成功！！！
第五回·发矫诏诸镇应曹公  破关兵三英战吕布 保存成功！！！
第六回·焚金阙董卓行凶    匿玉玺孙坚背约 保存成功！！！
第七回·袁绍磐河战公孙    孙坚跨江击刘表 保存成功！！！
第八回·王司徒巧使连环计  董太师大闹凤仪亭 保存成功！！！
第九回·除暴凶吕布助司徒  犯长安李傕听贾诩 保存成功！！！
第一十回·勤王室马腾举义    报父仇曹操兴师 保存成功！！！
第十一回·刘皇叔北海救孔融  吕温侯濮阳破曹操 保存成功！！！
第十二回·陶恭祖三让徐州    曹孟德大战吕布 保存成功！！！
第十三回·李傕郭汜大交兵  杨奉董承双救驾 保存成功！！！
第十四回·曹孟德移驾幸许都  吕奉先乘夜袭徐郡 保存成功！！！
第十五回·太史慈酣斗小霸王  孙伯符大战严白虎 保存成功！！！
第十六回·吕奉先射戟辕门    曹孟德败师淯水 保存成功！！！
第十七回·袁公路大起七军    曹孟德会合三将 保存成功！！！
第十八回·贾文和料敌决胜    夏侯惇拔矢啖睛 保存成功！！！
第十九回·下邳城曹操鏖兵    白门楼吕布殒命 保存成功！！！
第二十回·曹阿瞒许田打围    董国舅内阁受诏 保存成功！！！
第二十一回·曹操煮酒论英雄  关公赚城斩车胄 保存成功！！！
第二十二回·袁曹各起马步三军  关张共擒王刘二将 保存成功！！！
第二十三回·祢正平裸衣骂贼    吉太医下毒遭刑 保存成功！！！
第二十四回·国贼行凶杀贵妃    皇叔败走投袁绍 保存成功！！！
第二十五回·屯土山关公约三事  救白马曹操解重围 保存成功！！！
第二十六回·袁本初败兵折将    关云长挂印封金 保存成功！！！
第二十七回·美髯公千里走单骑  汉寿侯五关斩六将 保存成功！！！
第二十八回·斩蔡阳兄弟释疑    会古城主臣聚义 保存成功！！！
第二十九回·小霸王怒斩于吉    碧眼儿坐领江东 保存成功！！！
第三十回·战官渡本初败绩  劫乌巢孟德烧粮 保存成功！！！
第三十一回·曹操仓亭破本初    玄德荆州依刘表 保存成功！！！
第三十二回·夺冀州袁尚争锋

### xpath解析
- 环境安装：
    - pip install lxml
- 解析原理:html标签是以树状的形式进行展示
    - 1.实例化一个etree的对象，且将待解析的页面源码数据加载到该对象中
    - 2.调用etree对象的xpath方法结合着不同的xpath表达式实现标签的定位和数据提取
- 实例化etree对象
    - etree.parse('filename'):将本地html文档加载到该对象中
    - etree.HTML(page_text):网站获取的页面数据加载到该对象
- 标签定位：
    - 最左侧的/:如果xpath表达式最左侧是以/开头则表示该xpath表达式一定要从根标签开始定位指定标签(忽略)
    - 非最左侧的/:表示一个层级
    - 非左侧的//:表示多个层级
    - 最左侧的//：xpath表达式可以从任意位置进行标签定位
    
    - 属性定位：tagName[@attrName="value"]
    - 索引定位：tag[index]:索引是从1开始
    - 模糊匹配：
        - //div[contains(@class, "ng")]
        - //div[starts-with(@class, "ta")]
- 取文本
    - /text():直系文本内容
    - //text()：所有的文本内容
- 取属性
    - /@attrName

from lxml import etree
tree = etree.parse('test.html')
tree.xpath('/html/head/meta') #定位meta
tree.xpath('/html//meta') #定位meta
tree.xpath('/html//meta')#定位meta
tree.xpath('//meta') #定位meta

In [38]:
#定位class为song的div下面所有的p
tree.xpath('//div[@class="song"]/p')

[<Element p at 0x106c9aec8>,
 <Element p at 0x1071e4c08>,
 <Element p at 0x1071e4508>,
 <Element p at 0x1071e46c8>]

In [44]:
#定位class为song的div下面第2个p
tree.xpath('//div[@class="song"]/p[1]//text()')

['李清照']

In [46]:
tree.xpath('//a[@id="feng"]/@href')

['http://www.haha.com']

- 使用xpath爬取图片名称和图片数据
    - http://pic.netbian.com/4kmeinv/

- 局部数据解析：
    - 我们要将定位到的页面中的标签作为待解析的数据。再次使用xpath表达式解析待解析的数据。
    - 在局部数据解析的时候，xpath表达式中要使用./的操作，./表示的就是当前的局部数据(xpath的调用者)。

In [52]:
#爬取第一页的
dirName = 'GirlsLib'
if not os.path.exists(dirName):
    os.mkdir(dirName)

url = 'http://pic.netbian.com/4kmeinv/'
response = requests.get(url=url,headers=headers)
response.encoding = 'gbk'
page_text = response.text

#图片名称+图片数据
tree = etree.HTML(page_text)
#存储的是定位到的指定的li标签
li_list = tree.xpath('//div[@class="slist"]/ul/li')
for li in li_list:
#     print(type(li)) #li的数据类型和tree的数据类型一样，li也可以调用xpath方法
    title = li.xpath('./a/img/@alt')[0]+'.jpg'#进行局部数据解析
    img_src = 'http://pic.netbian.com'+li.xpath('./a/img/@src')[0]
    img_data = requests.get(url=img_src,headers=headers).content
    imgPath = dirName +'/'+title
    with open(imgPath,'wb') as fp:
        fp.write(img_data)
    print(title,'保存成功！！！')

糖果性感美女4k壁纸.jpg 保存成功！！！
陆萱萱 黑色丝袜美腿 养眼好看身材4k美女壁纸.jpg 保存成功！！！
刘奕宁Lynn 白色衣服 黑色短裙 长发清新美女4K壁纸.jpg 保存成功！！！
居家小清新白色小吊带美女4k壁纸.jpg 保存成功！！！
陆萱萱 制服黑色丝袜长腿美女4k壁纸.jpg 保存成功！！！
糖果 美女模特4k壁纸.jpg 保存成功！！！
美女模特制服连衣裙黑色丝袜长腿高跟鞋4k美女壁纸.jpg 保存成功！！！
白色婚纱裙子 白色丝袜美腿美女4k壁纸.jpg 保存成功！！！
收银员制服美女陆萱萱4k壁纸3840x2160.jpg 保存成功！！！
美女陆萱萱4k壁纸3840x2160.jpg 保存成功！！！
美脚美腿美足美女4k壁纸3840x2160.jpg 保存成功！！！
居家 一个人 长发美女 美腿 沙发 4k美女壁纸3840x2160.jpg 保存成功！！！
ceci美女模特4k壁纸.jpg 保存成功！！！
西装短裙女神杨紫嫣4k壁纸.jpg 保存成功！！！
杨紫嫣4k美女壁纸3840x2160.jpg 保存成功！！！
成果诸葛大力4k高清壁纸3840x2160.jpg 保存成功！！！
成果高清4k壁纸3840x2160.jpg 保存成功！！！
居家 沙发 养眼美女 白色裙子 白色丝袜美腿4k美女壁纸3840x2160.jpg 保存成功！！！
居家好身材养眼美女4k壁纸3840x2160.jpg 保存成功！！！
腿模Rubis 大波浪卷发美女 连衣包臀裙美腿屁股4k壁纸3840x2160.jpg 保存成功！！！
绿色植物 清新养眼美女4k壁纸3840x2160.jpg 保存成功！！！


In [54]:
#爬取多页
dirName = 'GirlsLib'
if not os.path.exists(dirName):
    os.mkdir(dirName)
    
#定义一个通用的url模板:不可变
url = 'http://pic.netbian.com/4kmeinv/index_%d.html'
for page in range(1,6):
    if page == 1:
        new_url = 'http://pic.netbian.com/4kmeinv/'
    else:
        new_url = format(url%page)
    response = requests.get(url=new_url,headers=headers)
    response.encoding = 'gbk'
    page_text = response.text

    #图片名称+图片数据
    tree = etree.HTML(page_text)
    #存储的是定位到的指定的li标签
    li_list = tree.xpath('//div[@class="slist"]/ul/li')
    for li in li_list:
    #     print(type(li)) #li的数据类型和tree的数据类型一样，li也可以调用xpath方法
        title = li.xpath('./a/img/@alt')[0]+'.jpg'#进行局部数据解析
        img_src = 'http://pic.netbian.com'+li.xpath('./a/img/@src')[0]
        img_data = requests.get(url=img_src,headers=headers).content
        imgPath = dirName +'/'+title
        with open(imgPath,'wb') as fp:
            fp.write(img_data)
        print(title,'保存成功！！！')

糖果性感美女4k壁纸.jpg 保存成功！！！
陆萱萱 黑色丝袜美腿 养眼好看身材4k美女壁纸.jpg 保存成功！！！
刘奕宁Lynn 白色衣服 黑色短裙 长发清新美女4K壁纸.jpg 保存成功！！！
居家小清新白色小吊带美女4k壁纸.jpg 保存成功！！！
陆萱萱 制服黑色丝袜长腿美女4k壁纸.jpg 保存成功！！！
糖果 美女模特4k壁纸.jpg 保存成功！！！
美女模特制服连衣裙黑色丝袜长腿高跟鞋4k美女壁纸.jpg 保存成功！！！
白色婚纱裙子 白色丝袜美腿美女4k壁纸.jpg 保存成功！！！
收银员制服美女陆萱萱4k壁纸3840x2160.jpg 保存成功！！！
美女陆萱萱4k壁纸3840x2160.jpg 保存成功！！！
美脚美腿美足美女4k壁纸3840x2160.jpg 保存成功！！！
居家 一个人 长发美女 美腿 沙发 4k美女壁纸3840x2160.jpg 保存成功！！！
ceci美女模特4k壁纸.jpg 保存成功！！！
西装短裙女神杨紫嫣4k壁纸.jpg 保存成功！！！
杨紫嫣4k美女壁纸3840x2160.jpg 保存成功！！！
成果诸葛大力4k高清壁纸3840x2160.jpg 保存成功！！！
成果高清4k壁纸3840x2160.jpg 保存成功！！！
居家 沙发 养眼美女 白色裙子 白色丝袜美腿4k美女壁纸3840x2160.jpg 保存成功！！！
居家好身材养眼美女4k壁纸3840x2160.jpg 保存成功！！！
腿模Rubis 大波浪卷发美女 连衣包臀裙美腿屁股4k壁纸3840x2160.jpg 保存成功！！！
绿色植物 清新养眼美女4k壁纸3840x2160.jpg 保存成功！！！
穿白色裙子的美女 看书 4k美女壁纸.jpg 保存成功！！！
白色衬衫长发清纯美女4k壁纸3840x2160.jpg 保存成功！！！
美女旅拍4k壁纸3840x2160.jpg 保存成功！！！
许诺连衣裙美女4k高清壁纸3840x2160.jpg 保存成功！！！
美女许诺居家写真4k高清壁纸3840x2160.jpg 保存成功！！！
冬季闭上眼睛的美女围巾4k壁纸3840x2160.jpg 保存成功！！！
写字的清纯美女4k壁纸3840x2160.jpg 保存成功！！！
制服美女 美腿 慕羽茜4K美女壁纸 3840x2160.jpg 保存成功！！！
冬季穿

- 需求：要求解析出携带html标签的局部数据？
    - bs4,bs4在实现标签定位的时候返回的直接就是定位到标签对应的字符串数据

- xpath表达式如何更加具有通用性？
    - 在xpath表达式中使用管道符分割的作用，可以表示管道符左右两侧的子xpath表达式同时生效或者一个生效。

In [60]:
#将https://www.aqistudy.cn/historydata/所有的城市名称解析出来
url = 'https://www.aqistudy.cn/historydata/'
page_text = requests.get(url=url,headers=headers).text

tree = etree.HTML(page_text)
# hot_cities = tree.xpath('//div[@class="bottom"]/ul/li/a/text()')
# all_cities = tree.xpath('//div[@class="bottom"]/ul/div[2]/li/a/text()')
tree.xpath('//div[@class="bottom"]/ul/li/a/text() | //div[@class="bottom"]/ul/div[2]/li/a/text()')

['北京',
 '上海',
 '广州',
 '深圳',
 '杭州',
 '天津',
 '成都',
 '南京',
 '西安',
 '武汉',
 '阿坝州',
 '安康',
 '阿克苏地区',
 '阿里地区',
 '阿拉善盟',
 '阿勒泰地区',
 '安庆',
 '安顺',
 '鞍山',
 '克孜勒苏州',
 '安阳',
 '蚌埠',
 '白城',
 '保定',
 '北海',
 '宝鸡',
 '北京',
 '毕节',
 '博州',
 '百色',
 '白沙',
 '白山',
 '保山',
 '保亭',
 '包头',
 '本溪',
 '白银',
 '巴彦淖尔',
 '滨州',
 '巴中',
 '亳州',
 '长春',
 '承德',
 '成都',
 '常德',
 '昌都',
 '赤峰',
 '昌江',
 '昌吉州',
 '五家渠',
 '澄迈',
 '重庆',
 '常熟',
 '长沙',
 '楚雄州',
 '朝阳',
 '滁州',
 '郴州',
 '潮州',
 '常州',
 '长治',
 '崇左',
 '沧州',
 '池州',
 '定安',
 '丹东',
 '东方',
 '东莞',
 '德宏州',
 '大连',
 '大理州',
 '大庆',
 '大同',
 '定西',
 '大兴安岭地区',
 '黔南州',
 '德阳',
 '东营',
 '达州',
 '德州',
 '儋州',
 '鄂尔多斯',
 '恩施州',
 '鄂州',
 '防城港',
 '抚顺',
 '佛山',
 '阜新',
 '阜阳',
 '富阳',
 '福州',
 '抚州',
 '广安',
 '贵港',
 '果洛州',
 '桂林',
 '甘南州',
 '贵阳',
 '广元',
 '固原',
 '广州',
 '甘孜州',
 '赣州',
 '淮安',
 '淮北',
 '鹤壁',
 '海北州',
 '河池',
 '邯郸',
 '海东地区',
 '哈尔滨',
 '合肥',
 '黄冈',
 '鹤岗',
 '红河州',
 '怀化',
 '黑河',
 '呼和浩特',
 '海口',
 '呼伦贝尔',
 '葫芦岛',
 '海门',
 '哈密地区',
 '淮南',
 '黄南州',
 '海南州',
 '黄山',
 '衡水',
 '黄石',
 '和田地区',
 '海西州',
 '衡阳',
 '河源',
 '湖州',
 '汉中',
 '杭州'